# tf_guide_tensorflow_basics_002_tensors

- tensorはnumpy配列のような操作ができる。
- すべてのtensorはimmutableであり変更できないため、新しいものを作成することだけができる。

In [2]:
import tensorflow as tf
import numpy as np

## basics

- rank-0のテンソルはスカラーであり、軸を持たない。

In [4]:
# This will be an int32 tensor by default; see "dtypes" below.
rank_0_tensor = tf.constant(4)
print(rank_0_tensor)

tf.Tensor(4, shape=(), dtype=int32)


- rank-1のテンソルはベクトルであり、軸を１つもつ。

In [5]:
# Let's make this a float tensor.
rank_1_tensor = tf.constant([2.0, 3.0, 4.0])
print(rank_1_tensor)

tf.Tensor([2. 3. 4.], shape=(3,), dtype=float32)


- rank-2のテンソルは行列(matrix)であり、２つの軸を持つ。

In [6]:
# If you want to be specific, you can set the dtype (see below) at creation time
rank_2_tensor = tf.constant([[1, 2],
                             [3, 4],
                             [5, 6]], dtype=tf.float16)
print(rank_2_tensor)

tf.Tensor(
[[1. 2.]
 [3. 4.]
 [5. 6.]], shape=(3, 2), dtype=float16)


- テンソルはこれ以上の軸を持つこともできる。以下は３軸の例である。

In [7]:
# There can be an arbitrary number of
# axes (sometimes called "dimensions")
rank_3_tensor = tf.constant([
  [[0, 1, 2, 3, 4],
   [5, 6, 7, 8, 9]],
  [[10, 11, 12, 13, 14],
   [15, 16, 17, 18, 19]],
  [[20, 21, 22, 23, 24],
   [25, 26, 27, 28, 29]],])

print(rank_3_tensor)

tf.Tensor(
[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]]

 [[10 11 12 13 14]
  [15 16 17 18 19]]

 [[20 21 22 23 24]
  [25 26 27 28 29]]], shape=(3, 2, 5), dtype=int32)


- `np.array`もしくは`tensor.numpy`でtensorはnumpy配列に変換ができる。

In [8]:
np.array(rank_2_tensor)

array([[1., 2.],
       [3., 4.],
       [5., 6.]], dtype=float16)

In [9]:
rank_2_tensor.numpy()

array([[1., 2.],
       [3., 4.],
       [5., 6.]], dtype=float16)

- 基本的なTensorクラスは矩形であることを要求します。
  - つまり要素が同じサイズを持つことが要求される。
  - 例外として、RaggedTensorやSparseTensorは要素数が違うものを扱える特殊なテンソルです。

- テンソルには様々な演算が使えます。

In [12]:
a = tf.constant([[1, 2],
                 [3, 4]])
b = tf.constant([[1, 1],
                 [1, 1]]) # Could have also said `tf.ones([2,2])`

print(tf.add(a, b), "\n") # 要素毎の加算
print(tf.multiply(a, b), "\n") # 要素枚の掛け算
print(tf.matmul(a, b), "\n") # 行列積

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32) 

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32) 

tf.Tensor(
[[3 3]
 [7 7]], shape=(2, 2), dtype=int32) 



- 上記は以下でも同じことが可能です。

In [13]:
print(a + b, "\n") # element-wise addition
print(a * b, "\n") # element-wise multiplication
print(a @ b, "\n") # matrix multiplication

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32) 

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32) 

tf.Tensor(
[[3 3]
 [7 7]], shape=(2, 2), dtype=int32) 



- Tensorには算術演算だけでなく様々なオペレーションができます。

In [14]:
c = tf.constant([[4.0, 5.0], [10.0, 1.0]])

# Find the largest value
print(tf.reduce_max(c))
# Find the index of the largest value
print(tf.argmax(c))
# Compute the softmax
print(tf.nn.softmax(c))

tf.Tensor(10.0, shape=(), dtype=float32)
tf.Tensor([1 0], shape=(2,), dtype=int64)
tf.Tensor(
[[2.6894143e-01 7.3105854e-01]
 [9.9987662e-01 1.2339458e-04]], shape=(2, 2), dtype=float32)


## About shapes

- Tensorの仕様を調べるためにいくつかのアプローチがある。
  - `tensor.dtype` : 要素の型を調べる。
  - `tensor.ndim` : ランク数を調べる。
  - `tensor.shape` : 各軸の長さを調べる。
  - `tf.size(tensor).numpy()` : 要素の総数を調べる。

In [15]:
rank_4_tensor = tf.zeros([3, 2, 4, 5])

In [21]:
print("Type of every element:", rank_4_tensor.dtype)
print("Number of axes:", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along axis 0 of tensor:", rank_4_tensor.shape[0])
print("Elements along the last axis of tensor:", rank_4_tensor.shape[-1])
print("Total number of elements (3*2*4*5): ", tf.size(rank_4_tensor).numpy())

Type of every element: <dtype: 'float32'>
Number of axes: 4
Shape of tensor: (3, 2, 4, 5)
Elements along axis 0 of tensor: 3
Elements along the last axis of tensor: 5
Total number of elements (3*2*4*5):  120


- 各軸の意味は理解して操作すると、抽象的なイメージより分かりやすい。
- 例えば画像の場合、rank-4で以下のように使われることが多い。
  - `shape: (batch, width, height, channels)`

## Indexing
- numpyとほぼ同じindex操作ができます。
  - `0` originである。
  - `-1`などの負のindexは末尾を起点とすることができる。
  - `:`でスライスすることができる。`start:stop:step`の形式で、`step`を省略する場合は`step=1`の扱い。

In [23]:
rank_1_tensor = tf.constant([0, 1, 1, 2, 3, 5, 8, 13, 21, 34])
print(rank_1_tensor.numpy())

[ 0  1  1  2  3  5  8 13 21 34]


In [24]:
print("First:", rank_1_tensor[0].numpy())
print("Second:", rank_1_tensor[1].numpy())
print("Last:", rank_1_tensor[-1].numpy())
print("Everything:", rank_1_tensor[:].numpy())
print("Before 4:", rank_1_tensor[:4].numpy())
print("From 4 to the end:", rank_1_tensor[4:].numpy())
print("From 2, before 7:", rank_1_tensor[2:7].numpy())
print("Every other item:", rank_1_tensor[::2].numpy())
print("Reversed:", rank_1_tensor[::-1].numpy())

First: 0
Second: 1
Last: 34
Everything: [ 0  1  1  2  3  5  8 13 21 34]
Before 4: [0 1 1 2]
From 4 to the end: [ 3  5  8 13 21 34]
From 2, before 7: [1 2 3 5 8]
Every other item: [ 0  1  3  8 21]
Reversed: [34 21 13  8  5  3  2  1  1  0]


- multi-axis indexingも同様。

In [27]:
print(rank_2_tensor.numpy())

# Pull out a single value from a 2-rank tensor
print(rank_2_tensor[1, 1].numpy())

# Get row and column tensors
print("Second row:", rank_2_tensor[1, :].numpy())
print("Second column:", rank_2_tensor[:, 1].numpy())
print("Last row:", rank_2_tensor[-1, :].numpy())
print("First item in last column:", rank_2_tensor[0, -1].numpy())
print("Skip the first row:")
print(rank_2_tensor[1:, :].numpy(), "\n")

[[1. 2.]
 [3. 4.]
 [5. 6.]]
4.0
Second row: [3. 4.]
Second column: [2. 4. 6.]
Last row: [5. 6.]
First item in last column: 2.0
Skip the first row:
[[3. 4.]
 [5. 6.]] 



## Manipulating shapes

- Tensorのshapeを変換する方法。

In [28]:
# Shape returns a `TensorShape` object that shows the size along each axis
x = tf.constant([[1], [2], [3]])
print(x.shape)

(3, 1)


In [31]:
# You can convert this object into a Python list, too
print(x.shape.as_list())

[3, 1]


In [33]:
# You can reshape a tensor to a new shape.
# Note that you're passing in a list
reshaped = tf.reshape(x, [1, 3])

print(x.shape)
print(reshaped.shape)

(3, 1)
(1, 3)


In [34]:
print(rank_3_tensor)

tf.Tensor(
[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]]

 [[10 11 12 13 14]
  [15 16 17 18 19]]

 [[20 21 22 23 24]
  [25 26 27 28 29]]], shape=(3, 2, 5), dtype=int32)


- reshapeしてもデータはメモリ内のレイアウトを維持し、新しい形状のTensorを作成する。
- tensorflowはrow-major(行志向、右端のindexのインクリメントがメモリ上の1step)の配列となっている。
- flattenをすれば、それがメモリ上に並んでいることがわかる。

In [35]:
# A `-1` passed in the `shape` argument says "Whatever fits".
print(tf.reshape(rank_3_tensor, [-1]))

tf.Tensor(
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29], shape=(30,), dtype=int32)


- reshapeの合理的な使用方法は、隣接する軸を結合もしくは分割することだけです(これは確かに)。

In [37]:
print(tf.reshape(rank_3_tensor, [3*2, 5]), "\n")
print(tf.reshape(rank_3_tensor, [3, -1]))

tf.Tensor(
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]
 [25 26 27 28 29]], shape=(6, 5), dtype=int32) 

tf.Tensor(
[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]], shape=(3, 10), dtype=int32)


![](./img/tf_guide_tensorflow_basics_002_tensors_2022-03-13-19-42-34.png)

- reshapeは順序は変わらないため、軸の入れ替えはできません。
- 軸の入れ替えにはtransposeを使用します。
- 割り切れる場合は、reshapeに成功しますが、意味をなさないデータ構造となることがわかります。

In [39]:
print(rank_3_tensor)

# Bad examples: don't do this

# You can't reorder axes with reshape.
print(tf.reshape(rank_3_tensor, [2, 3, 5]), "\n") 

# This is a mess
print(tf.reshape(rank_3_tensor, [5, 6]), "\n")

# This doesn't work at all
try:
    tf.reshape(rank_3_tensor, [7, -1])
except Exception as e:
    print(f"{type(e).__name__}: {e}")

tf.Tensor(
[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]]

 [[10 11 12 13 14]
  [15 16 17 18 19]]

 [[20 21 22 23 24]
  [25 26 27 28 29]]], shape=(3, 2, 5), dtype=int32)
tf.Tensor(
[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]]

 [[15 16 17 18 19]
  [20 21 22 23 24]
  [25 26 27 28 29]]], shape=(2, 3, 5), dtype=int32) 

tf.Tensor(
[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]
 [24 25 26 27 28 29]], shape=(5, 6), dtype=int32) 

InvalidArgumentError: Input to reshape is a tensor with 30 values, but the requested shape requires a multiple of 7 [Op:Reshape]


![](./img/tf_guide_tensorflow_basics_002_tensors_2022-03-13-19-44-23.png)

## More on DTypes

- 型の指定は、`tf.constant`や`tf.cast`のdtype引数で行うことができる。

In [40]:
the_f64_tensor = tf.constant([2.2, 3.3, 4.4], dtype=tf.float64)
the_f16_tensor = tf.cast(the_f64_tensor, dtype=tf.float16)
# Now, cast to an uint8 and lose the decimal precision
the_u8_tensor = tf.cast(the_f16_tensor, dtype=tf.uint8)
print(the_u8_tensor)

tf.Tensor([2 3 4], shape=(3,), dtype=uint8)


## Broadcasting

- numpyと同様、Tensorでもbroadcastingを動作させることができる。
- numpyの時に調べたルールを張っておく。
  - AとBの演算したい場合に、shapeの長さが同じだと、1の部分の次元の要素を複製して演算できる。
    - shape=(4,3,2)とshape=(1,3,2)の演算はOK（左側に1があってもOK）
    - shape=(4,3,2)とshape=(4,3,1)の演算はOK（左側に1があってもOK）
    - shape=(5,4,3,2)とshape=(5,4,1,1)の演算はOK（1が複数あってもOK）
    - shape=(5,4,3,2)とshape=(1,4,1,1)の演算はOK（1が飛んでてもOK）
    - shape=(5,1,3,2)とshape=(5,4,1,2)の演算はOK（お互い違う場合でもOK）
  - また、shapeの長さが異なる場合でも、親側（左側）には1を自動追加することができる。
    - shape=(5,4,3,2)とshape=(4,3,2)の演算はOK（shape=(4,3,2)は勝手に(1,4,3,2)と解釈）
    - shape=(5,4,3,2)とshape=(3,2)の演算はOK（shape=(3,2)は勝手に(1,1,4,3,2)と解釈）
  - ちなみに、子側（右側）には1を自動追加できない。理由は両方に自動追加できると、複数の選択肢が発生するためと思われる。
    - shape=(5,4,3,2)とshape=(5,4)の演算はNG
    - やりたいときは、足りない側を明示的にreshapeしてあげる必要がある。

- 以下の例はこのように機能する。
  - 次元数が不一致だが、yの親側に1を足すことが可能なので、x.shape=(3,1), y.shape=(1,4)として扱われる。
  - x.shape=(3,1)は1の部分が4つに複製され、以下の通りとなる。
  ```
  [[1 1 1 1]
   [2 2 2 2]
   [3 3 3 3]]
  ```
  - y.shape=(1,4)は1の部分が3つに複製され、以下の通りとなる。
  ```
  [[1 2 3 4]
   [1 2 3 4]
   [1 2 3 4]]
  ```
  - この要素同士の掛け算が、計算結果となる。

In [41]:
# These are the same computations
x = tf.reshape(x,[3,1])
y = tf.range(1, 5)
print(x, "\n")
print(y, "\n")
print(tf.multiply(x, y))

tf.Tensor(
[[1]
 [2]
 [3]], shape=(3, 1), dtype=int32) 

tf.Tensor([1 2 3 4], shape=(4,), dtype=int32) 

tf.Tensor(
[[ 1  2  3  4]
 [ 2  4  6  8]
 [ 3  6  9 12]], shape=(3, 4), dtype=int32)


- broadcast処理は、要素を複製したTensorをメモリ上に実体化しないため、メモリ的にも処理量的にも効率が良い。
- broadcastは手動で以下のように展開を試すこともできる。

In [45]:
print(tf.broadcast_to(tf.constant([1, 2, 3]), [4, 3]))

tf.Tensor(
[[1 2 3]
 [1 2 3]
 [1 2 3]
 [1 2 3]], shape=(4, 3), dtype=int32)


## tf.convert_to_tensor
- `tf.matmul`や`tf.reshape`などのoperatorは、Tensorを入力とするが、Pythonのobjectを与えてもTensorに自動で変換する。
- これはtf.convert_to_tensorにより実際に変換されている。
- 変換のレジストリがあり、NumPyのndarray、TensorShape、Pythonのリスト、tf.Variableなどのほとんどのオブジェクトクラスはすべて自動的に変換されます。
- 以下に詳細な情報があり、自身で変換を定義したい場合などもこちらを参照してください。
  - https://www.tensorflow.org/api_docs/python/tf/register_tensor_conversion_function

## Ragged Tensors

- 要素数のことなるものを扱うための特殊なTensorです。
- 以下は通常のTensorでエラーになります。

In [61]:
ragged_list = [
    [0, 1, 2, 3],
    [4, 5],
    [6, 7, 8],
    [9]]

try:
    tensor = tf.constant(ragged_list)
except Exception as e:
    print(f"{type(e).__name__}: {e}")

ValueError: Can't convert non-rectangular Python sequence to Tensor.


- ragged tensorを使うには以下のようにします。

In [48]:
ragged_tensor = tf.ragged.constant(ragged_list)
print(ragged_tensor)

<tf.RaggedTensor [[0, 1, 2, 3], [4, 5], [6, 7, 8], [9]]>


- ragged tensorのshapeはNoneを含んだ以下のような形状となります。

In [55]:
print(ragged_tensor.shape)

(4, None)


- `tf.RaggedTensor.to_tensor()`により、0詰めされたTensorに変換することも可能です。

In [56]:
print(ragged_tensor.to_tensor())

tf.Tensor(
[[0 1 2 3]
 [4 5 0 0]
 [6 7 8 0]
 [9 0 0 0]], shape=(4, 4), dtype=int32)


- rankを調べるには、ndimsの代わりに、ragged_rankを使います。

In [58]:
ragged_tensor.ragged_rank

1

## String tensors

- dtypeが`tf.string`のTensorであり、Tensorの中に可変長のバイト配列データを格納したものである。
- `tf.string`はPythonのstrとことなり、indexをもちません。また文字列長はテンソルの軸ではありません。
- 文字列操作の詳細は以下を参照ください。
  - https://www.tensorflow.org/api_docs/python/tf/strings
- 以下はscalarなstring tensorの例です。

In [62]:
# Tensors can be strings, too here is a scalar string.
scalar_string_tensor = tf.constant("Gray wolf")
print(scalar_string_tensor)

tf.Tensor(b'Gray wolf', shape=(), dtype=string)


- ベクトルのようにも扱えます。

In [63]:
# If you have three string tensors of different lengths, this is OK.
tensor_of_strings = tf.constant(["Gray wolf",
                                 "Quick brown fox",
                                 "Lazy dog"])
# Note that the shape is (3,). The string length is not included.
print(tensor_of_strings)

tf.Tensor([b'Gray wolf' b'Quick brown fox' b'Lazy dog'], shape=(3,), dtype=string)


- 絵文字ももちろん扱えます。

In [64]:
tf.constant("🥳👍")

<tf.Tensor: shape=(), dtype=string, numpy=b'\xf0\x9f\xa5\xb3\xf0\x9f\x91\x8d'>

- 基本的な操作は、`tf.strings`として定義され、その一例としては、splitなどです。
- scalarの場合は、tensorとして結果が返ってきます。

In [65]:
# You can use split to split a string into a set of tensors
print(tf.strings.split(scalar_string_tensor, sep=" "))

tf.Tensor([b'Gray' b'wolf'], shape=(2,), dtype=string)


- 一方で、ベクトルの場合のsplitは要素数が異なるので、RaggedTensorで返ってきます。

In [66]:
# ...but it turns into a `RaggedTensor` if you split up a tensor of strings,
# as each string might be split into a different number of parts.
print(tf.strings.split(tensor_of_strings))

<tf.RaggedTensor [[b'Gray', b'wolf'], [b'Quick', b'brown', b'fox'], [b'Lazy', b'dog']]>


- `split`と`to_number`を組み合わせて、文字列を数値列に変換するなどの処理も可能です。

In [67]:
text = tf.constant("1 10 100")
print(tf.strings.to_number(tf.strings.split(text, " ")))

tf.Tensor([  1.  10. 100.], shape=(3,), dtype=float32)


- `tf.cast`などを用いて、string tensorを数値列に変換することはできませんが、文字列を1byteずつに変換し、それを数値列にすることは可能です。

In [69]:
byte_strings = tf.strings.bytes_split(tf.constant("Duck"))
byte_ints = tf.io.decode_raw(tf.constant("Duck"), tf.uint8)
print("Byte strings:", byte_strings)
print("Bytes:", byte_ints)

Byte strings: tf.Tensor([b'D' b'u' b'c' b'k'], shape=(4,), dtype=string)
Bytes: tf.Tensor([ 68 117  99 107], shape=(4,), dtype=uint8)


- マルチバイト文字を、その文字単位で分割する操作なども可能です。

In [70]:
# Or split it up as unicode and then decode it
unicode_bytes = tf.constant("アヒル 🦆")
unicode_char_bytes = tf.strings.unicode_split(unicode_bytes, "UTF-8")
unicode_values = tf.strings.unicode_decode(unicode_bytes, "UTF-8")

print("\nUnicode bytes:", unicode_bytes)
print("\nUnicode chars:", unicode_char_bytes)
print("\nUnicode values:", unicode_values)


Unicode bytes: tf.Tensor(b'\xe3\x82\xa2\xe3\x83\x92\xe3\x83\xab \xf0\x9f\xa6\x86', shape=(), dtype=string)

Unicode chars: tf.Tensor([b'\xe3\x82\xa2' b'\xe3\x83\x92' b'\xe3\x83\xab' b' ' b'\xf0\x9f\xa6\x86'], shape=(5,), dtype=string)

Unicode values: tf.Tensor([ 12450  12498  12523     32 129414], shape=(5,), dtype=int32)


- `tf.string`はすべてのraw bytesのデータとして使われます。
- 画像やcsvをパースするなどの処理は、`tf.io`モジュールを使います。

## Sparse tensors

- 要素がほぼ0となるようなテンソルを扱うためのsparse tensorを使うことができる。
- これは自然言語処理などでは頻繁に発生する。
- sparse tensorでは、非ゼロのindexとその値をデータとして扱うことで、メモリ上効率的にとり扱うことのできるデータ型である。

In [72]:
# Sparse tensors store values by index in a memory-efficient manner
sparse_tensor = tf.sparse.SparseTensor(indices=[[0, 0], [1, 2]],
                                       values=[1, 2],
                                       dense_shape=[3, 4])
print(sparse_tensor, "\n")

# You can convert sparse tensors to dense
print(tf.sparse.to_dense(sparse_tensor))

SparseTensor(indices=tf.Tensor(
[[0 0]
 [1 2]], shape=(2, 2), dtype=int64), values=tf.Tensor([1 2], shape=(2,), dtype=int32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64)) 

tf.Tensor(
[[1 0 0 0]
 [0 0 2 0]
 [0 0 0 0]], shape=(3, 4), dtype=int32)


In [73]:
# 以上